# Users motivations

Finally, to give some insights about the why question, I did a manual analysis of the top 12 most contributors as well as people that have more weaponising edits than not weaponising edits. The manual analysis is based on the User page and the User talk page mainly. We are seeking banners, templates and semantic behaviour that tells a lot about someone’s identity, ideology and culture. However, those features can give some insights and some potential 
direction but we cannot and we won’t determine someone's culture only based on his wikipedia page. 


## Templates analysis

This section is a depp analysis into the users template banner used. Templates are very often used by user as : 
* identity markers
* ideological broadcasting
* group affiliations signals
* defensive framings

In [ ]:
import requests
import mwparserfromhell
import csv
import json
import time
from tqdm import tqdm

URL = "https://en.wikipedia.org/w/api.php"
HEADERS = {
    "User-Agent": "DH_Project/1.0 (https://www.epfl.ch/labs/dhlab/; maxime.garambois@epfl.ch)"
}

SLEEP_TIME = 0.5  # be polite to the API


def fetch_wikitext(title):
    params = {
        "action": "query",
        "prop": "revisions",
        "rvprop": "content",
        "rvslots": "main",
        "titles": title,
        "format": "json",
        "formatversion": "2"
    }

    r = requests.get(URL, params=params, headers=HEADERS)
    r.raise_for_status()
    data = r.json()

    pages = data.get("query", {}).get("pages", [])
    if not pages or "missing" in pages[0]:
        return None

    revisions = pages[0].get("revisions", [])
    if not revisions:
        return None

    return revisions[0]["slots"]["main"]["content"]


def extract_templates(wikitext):
    wikicode = mwparserfromhell.parse(wikitext)
    templates = []

    for tpl in wikicode.filter_templates(recursive=True):
        templates.append({
            "template_name": str(tpl.name).strip(),
            "parameters": {
                str(param.name).strip(): str(param.value).strip()
                for param in tpl.params
            }
        })

    return templates


def process_user(username):
    results = []

    for page_type, title in [
        ("user", f"User:{username}"),
        ("user_talk", f"User talk:{username}")
    ]:
        try:
            wikitext = fetch_wikitext(title)
            if not wikitext:
                continue

            templates = extract_templates(wikitext)

            for tpl in templates:
                results.append({
                    "username": username,
                    "page_type": page_type,
                    "template_name": tpl["template_name"],
                    "parameters": tpl["parameters"]
                })

        except Exception as e:
            print(f"[ERROR] {username} ({page_type}): {e}")

        time.sleep(SLEEP_TIME)

    return results


def run():
    all_results = []

    for user in tqdm(top10_users, desc="Processing users"):
        all_results.extend(process_user(user))

    # Save JSON (full fidelity)
    with open("user_templates_raw.json", "w", encoding="utf-8") as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2)

    # Save CSV (easy inspection)
    with open("user_templates_raw.csv", "w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=["username", "page_type", "template_name", "parameters"]
        )
        writer.writeheader()
        for row in all_results:
            writer.writerow(row)

    print("✔ Extraction complete")

In [ ]:
df_templates = pd.read_csv('user_templates_raw.csv')